# Lab 3.4 - Visual Agent Building with Langflow + watsonx.ai

This notebook demonstrates how to build AI agents visually using Langflow with watsonx.ai integration.

## What is Langflow?

Langflow is a **visual UI for LangChain** that allows you to build complex agent workflows through a drag-and-drop interface. It's perfect for:

- Rapid prototyping of agent workflows
- Visual debugging of LLM pipelines  
- Collaboration between technical and non-technical teams
- Testing different agent architectures quickly

## What You'll Learn

- Installing and running Langflow
- Integrating watsonx.ai with Langflow
- Building visual agent workflows
- Creating RAG pipelines visually
- Exporting and deploying Langflow flows
- Using Langflow programmatically

## Architecture

```
Langflow UI (Browser)
     |
     v
Visual Flow Builder
  - Drag & Drop Components
  - Connect Nodes
  - Configure watsonx.ai
     |
     v
LangChain Backend
  - watsonx.ai LLM
  - Custom Tools
  - Agent Logic
     |
     v
Execution & Results
```

---

## 1. Setup and Installation

### Google Colab Compatibility

This notebook works in both Google Colab and local environments.

In [ ]:
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
    print("\nNote: Langflow UI will be accessible through a public URL in Colab")
except ImportError:
    IN_COLAB = False
    print("✓ Running in local environment")
    print("\nNote: Langflow UI will be accessible at http://localhost:7860")

### Install Langflow and Dependencies

In [ ]:
# Install Langflow and required packages
!pip install -q langflow "ibm-watsonx-ai>=1.1.22" langchain-ibm requests

print("✓ Langflow and dependencies installed successfully")

## 2. Configure watsonx.ai Credentials

Set up your IBM watsonx.ai credentials for use in Langflow.

In [ ]:
import os
from getpass import getpass

# Configuration for watsonx.ai
WATSONX_URL = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com")

if not os.getenv("WATSONX_APIKEY"):
    WATSONX_APIKEY = getpass("Enter your watsonx.ai API Key: ")
    os.environ["WATSONX_APIKEY"] = WATSONX_APIKEY
else:
    WATSONX_APIKEY = os.getenv("WATSONX_APIKEY")

if not os.getenv("WATSONX_PROJECT_ID"):
    WATSONX_PROJECT_ID = getpass("Enter your watsonx.ai Project ID: ")
    os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID
else:
    WATSONX_PROJECT_ID = os.getenv("WATSONX_PROJECT_ID")

# Set environment variables for Langflow to use
os.environ["WATSONX_URL"] = WATSONX_URL

# Model configuration
LLM_MODEL_ID = os.getenv("LLM_MODEL_ID", "ibm/granite-3-8b-instruct")

print("✓ Configuration loaded and environment variables set")
print(f"  Model: {LLM_MODEL_ID}")
print(f"  URL: {WATSONX_URL}")

## 3. Start Langflow Server

We'll start the Langflow server in the background.

In [ ]:
import subprocess
import time
import threading

def start_langflow_background():
    """
    Start Langflow server in the background.
    """
    if IN_COLAB:
        # In Colab, use pyngrok to create a public URL
        !pip install -q pyngrok
        from pyngrok import ngrok
        
        # Start Langflow
        proc = subprocess.Popen(
            ["langflow", "run", "--host", "0.0.0.0", "--port", "7860"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        # Wait for server to start
        time.sleep(10)
        
        # Create public URL
        public_url = ngrok.connect(7860)
        print(f"\n🌐 Langflow is running at: {public_url}")
        print("\nClick the link above to open Langflow in a new tab")
        
        return proc, public_url
    else:
        # Local environment
        proc = subprocess.Popen(
            ["langflow", "run", "--host", "127.0.0.1", "--port", "7860"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        time.sleep(5)
        
        print("\n🌐 Langflow is running at: http://localhost:7860")
        print("\nOpen this URL in your browser to access Langflow")
        
        return proc, "http://localhost:7860"

# Start Langflow
print("Starting Langflow server...")
print("This may take a moment...\n")

langflow_proc, langflow_url = start_langflow_background()

print("\n" + "=" * 80)
print("LANGFLOW SERVER STARTED")
print("=" * 80)

## 4. Using Langflow with watsonx.ai

### Step-by-Step Guide

#### 1. Open Langflow UI

- Click on the Langflow URL from the cell above
- The Langflow interface will open in a new tab

#### 2. Create a New Flow

- Click "New Project" or "+ New Flow"
- You'll see a blank canvas with a sidebar of components

#### 3. Add watsonx.ai LLM Component

Since watsonx.ai might not have a pre-built component, we'll use a custom approach:

1. **Option A: Use LangChain LLM Component**
   - Drag "LLM" from the sidebar
   - Configure with custom code (see below)

2. **Option B: Use Custom Component**
   - Create a custom Python component
   - Import watsonx.ai integration

#### 4. Build Your Flow

Connect components visually:
- Input → LLM → Output (simple flow)
- Input → RAG Tool → LLM → Output (RAG flow)
- Input → Agent → Tools → LLM → Output (agent flow)

---

## 5. Sample Flow Configurations

Here are some pre-built flow configurations you can import into Langflow.

### Flow 1: Simple Question Answering with watsonx.ai

In [ ]:
import json

# Simple QA Flow Configuration
simple_qa_flow = {
    "name": "Simple QA with watsonx.ai",
    "description": "A basic question answering flow using watsonx.ai",
    "nodes": [
        {
            "id": "input",
            "type": "ChatInput",
            "position": {"x": 100, "y": 200}
        },
        {
            "id": "llm",
            "type": "CustomLLM",
            "config": {
                "model_id": LLM_MODEL_ID,
                "api_key": "${WATSONX_APIKEY}",
                "project_id": "${WATSONX_PROJECT_ID}",
                "url": WATSONX_URL
            },
            "position": {"x": 400, "y": 200}
        },
        {
            "id": "output",
            "type": "ChatOutput",
            "position": {"x": 700, "y": 200}
        }
    ],
    "edges": [
        {"source": "input", "target": "llm"},
        {"source": "llm", "target": "output"}
    ]
}

# Save flow configuration
with open("langflow_simple_qa.json", "w") as f:
    json.dump(simple_qa_flow, f, indent=2)

print("✓ Simple QA flow configuration saved to: langflow_simple_qa.json")
print("\nYou can import this file in Langflow using: Import → Upload JSON")

### Flow 2: RAG Pipeline with watsonx.ai

In [ ]:
# RAG Flow Configuration
rag_flow = {
    "name": "RAG Pipeline with watsonx.ai",
    "description": "Retrieval-Augmented Generation using watsonx.ai",
    "nodes": [
        {
            "id": "input",
            "type": "ChatInput",
            "position": {"x": 100, "y": 200}
        },
        {
            "id": "retriever",
            "type": "APIRequest",
            "config": {
                "url": "${ACCELERATOR_API_URL}",
                "method": "POST"
            },
            "position": {"x": 300, "y": 200}
        },
        {
            "id": "prompt",
            "type": "PromptTemplate",
            "config": {
                "template": "Context: {context}\n\nQuestion: {question}\n\nAnswer:"
            },
            "position": {"x": 500, "y": 200}
        },
        {
            "id": "llm",
            "type": "CustomLLM",
            "config": {
                "model_id": LLM_MODEL_ID,
                "api_key": "${WATSONX_APIKEY}",
                "project_id": "${WATSONX_PROJECT_ID}"
            },
            "position": {"x": 700, "y": 200}
        },
        {
            "id": "output",
            "type": "ChatOutput",
            "position": {"x": 900, "y": 200}
        }
    ],
    "edges": [
        {"source": "input", "target": "retriever"},
        {"source": "retriever", "target": "prompt", "data": "context"},
        {"source": "input", "target": "prompt", "data": "question"},
        {"source": "prompt", "target": "llm"},
        {"source": "llm", "target": "output"}
    ]
}

# Save flow configuration
with open("langflow_rag_pipeline.json", "w") as f:
    json.dump(rag_flow, f, indent=2)

print("✓ RAG pipeline flow configuration saved to: langflow_rag_pipeline.json")
print("\nYou can import this file in Langflow using: Import → Upload JSON")

## 6. Custom watsonx.ai Component for Langflow

Create a custom component that can be used in Langflow.

In [ ]:
# Custom watsonx.ai component code
watsonx_component_code = '''
from langflow import CustomComponent
from langchain_ibm import WatsonxLLM
from langflow.field_typing import Text

class WatsonxAIComponent(CustomComponent):
    """
    Custom Langflow component for IBM watsonx.ai.
    """
    
    display_name = "watsonx.ai LLM"
    description = "IBM watsonx.ai foundation models"
    
    def build_config(self):
        return {
            "model_id": {
                "display_name": "Model ID",
                "info": "watsonx.ai model identifier",
                "value": "ibm/granite-3-8b-instruct"
            },
            "api_key": {
                "display_name": "API Key",
                "info": "IBM Cloud API Key",
                "password": True
            },
            "project_id": {
                "display_name": "Project ID",
                "info": "watsonx.ai Project ID"
            },
            "url": {
                "display_name": "URL",
                "info": "watsonx.ai endpoint URL",
                "value": "https://us-south.ml.cloud.ibm.com"
            },
            "temperature": {
                "display_name": "Temperature",
                "info": "Sampling temperature (0-1)",
                "value": 0.3
            },
            "max_new_tokens": {
                "display_name": "Max New Tokens",
                "info": "Maximum tokens to generate",
                "value": 1000
            }
        }
    
    def build(self, 
              model_id: str,
              api_key: str,
              project_id: str,
              url: str,
              temperature: float = 0.3,
              max_new_tokens: int = 1000) -> WatsonxLLM:
        """
        Build and return a watsonx.ai LLM instance.
        """
        params = {
            "decoding_method": "greedy",
            "max_new_tokens": max_new_tokens,
            "temperature": temperature
        }
        
        llm = WatsonxLLM(
            model_id=model_id,
            url=url,
            apikey=api_key,
            project_id=project_id,
            params=params
        )
        
        return llm
'''

# Save custom component
with open("watsonx_component.py", "w") as f:
    f.write(watsonx_component_code)

print("✓ Custom watsonx.ai component saved to: watsonx_component.py")
print("\nTo use this component in Langflow:")
print("1. Copy the file to Langflow's custom_components directory")
print("2. Restart Langflow")
print("3. The component will appear in the sidebar under 'Custom'")

## 7. Programmatic Access to Langflow

You can also interact with Langflow flows programmatically using the API.

In [ ]:
import requests
import json

def run_langflow_flow(flow_id: str, input_text: str, langflow_url: str = "http://localhost:7860"):
    """
    Run a Langflow flow programmatically.
    
    Args:
        flow_id: The ID of the flow to run
        input_text: The input text for the flow
        langflow_url: Base URL of the Langflow server
    
    Returns:
        Response from the flow
    """
    api_url = f"{langflow_url}/api/v1/run/{flow_id}"
    
    payload = {
        "inputs": {
            "input": input_text
        }
    }
    
    try:
        response = requests.post(api_url, json=payload)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        return {"error": str(e)}


# Example usage (replace with your actual flow ID)
# result = run_langflow_flow(
#     flow_id="your-flow-id-here",
#     input_text="What is watsonx.ai?"
# )
# print(json.dumps(result, indent=2))

print("✓ Langflow API helper function defined")

## 8. Example Flows to Build

Here are some example flows you can build in Langflow:

### Flow 1: Simple Chatbot

**Components:**
1. Chat Input
2. watsonx.ai LLM
3. Chat Output

**Flow:**
```
Chat Input → watsonx.ai LLM → Chat Output
```

**Use Case:** Basic conversational AI

---

### Flow 2: RAG System

**Components:**
1. Chat Input
2. Document Loader / API Call (RAG service)
3. Prompt Template
4. watsonx.ai LLM
5. Chat Output

**Flow:**
```
Chat Input → RAG Service → Prompt Template → watsonx.ai LLM → Chat Output
```

**Use Case:** Question answering over documents

---

### Flow 3: Multi-Tool Agent

**Components:**
1. Chat Input
2. Tool Router (custom)
3. RAG Tool
4. Calculator Tool
5. watsonx.ai LLM
6. Chat Output

**Flow:**
```
Chat Input → Router → [RAG Tool / Calculator Tool] → watsonx.ai LLM → Chat Output
```

**Use Case:** Agent with multiple capabilities

---

### Flow 4: Conversational Memory

**Components:**
1. Chat Input
2. Conversation Buffer Memory
3. Prompt Template (with history)
4. watsonx.ai LLM
5. Chat Output

**Flow:**
```
Chat Input → Memory → Prompt Template → watsonx.ai LLM → Chat Output
                ↑                                              |
                +----------------------------------------------+
```

**Use Case:** Chatbot with conversation history

## 9. Best Practices for Langflow

### Design Principles

1. **Start Simple**: Begin with basic flows and add complexity gradually
2. **Modular Components**: Create reusable components for common operations
3. **Test Incrementally**: Test each node individually before connecting them
4. **Use Logging**: Add logging nodes to debug flow execution
5. **Version Control**: Export and save your flows regularly

### Performance Tips

1. **Caching**: Use caching for expensive operations (embeddings, API calls)
2. **Batch Processing**: Process multiple inputs together when possible
3. **Async Execution**: Use async components for I/O-bound operations
4. **Resource Limits**: Set appropriate limits for token generation
5. **Error Handling**: Add error handling nodes for robustness

### Security Considerations

1. **API Keys**: Store credentials as environment variables
2. **Input Validation**: Validate user inputs before processing
3. **Rate Limiting**: Implement rate limits for public endpoints
4. **Access Control**: Restrict access to sensitive flows
5. **Audit Logs**: Track flow executions for security monitoring

## 10. Exporting and Deploying Flows

### Export Options

1. **JSON Export**
   - Export flow as JSON for version control
   - Share flows with team members
   - Import into other Langflow instances

2. **Python Code Export**
   - Generate standalone Python code
   - Integrate into existing applications
   - Customize and extend functionality

3. **API Endpoint**
   - Deploy flow as REST API
   - Access from any application
   - Scale independently

### Deployment Strategies

1. **Local Development**: Run Langflow locally for testing
2. **Cloud Deployment**: Deploy to cloud platforms (AWS, GCP, Azure)
3. **Container Deployment**: Package as Docker container
4. **Serverless**: Deploy as serverless function

## 11. Integrating with Other Tools

Langflow can integrate with various tools and services:

In [ ]:
# Example integrations

integration_examples = {
    "Vector Databases": [
        "Weaviate",
        "Pinecone",
        "Chroma",
        "Milvus"
    ],
    "Document Loaders": [
        "PDF Loader",
        "Web Scraper",
        "CSV Loader",
        "API Connector"
    ],
    "LLM Providers": [
        "watsonx.ai (IBM)",
        "OpenAI",
        "Hugging Face",
        "Anthropic"
    ],
    "Tools": [
        "Google Search",
        "Wikipedia",
        "Python REPL",
        "Custom APIs"
    ],
    "Memory": [
        "Conversation Buffer",
        "Entity Memory",
        "Vector Store Memory",
        "Redis Cache"
    ]
}

print("Available Integrations in Langflow:\n")
for category, items in integration_examples.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  - {item}")

## 12. Troubleshooting Common Issues

### Issue 1: Langflow won't start

**Solution:**
```python
# Check if port 7860 is already in use
!lsof -ti:7860 | xargs kill -9  # Kill any process on port 7860
# Then restart Langflow
```

### Issue 2: watsonx.ai authentication fails

**Solution:**
- Verify API key and Project ID are correct
- Check that environment variables are set
- Ensure API key has proper permissions

### Issue 3: Flow execution is slow

**Solutions:**
- Reduce max_new_tokens parameter
- Use caching for repeated operations
- Optimize prompt templates
- Use faster models for non-critical components

### Issue 4: Custom components not appearing

**Solution:**
- Restart Langflow after adding custom components
- Check component code for syntax errors
- Verify component is in correct directory
- Check Langflow logs for error messages

## 13. Summary and Key Takeaways

### What We Learned

1. **Visual Development**: Build AI agents using drag-and-drop interface
2. **watsonx.ai Integration**: Connect IBM Granite models to Langflow
3. **Custom Components**: Create reusable components for specific needs
4. **Flow Export**: Export flows as JSON or Python code
5. **API Access**: Interact with flows programmatically

### Key Benefits of Langflow

| Feature | Benefit |
|---------|----------|
| Visual Interface | Faster prototyping and development |
| No-Code/Low-Code | Accessible to non-programmers |
| Real-time Testing | Immediate feedback on changes |
| Component Reuse | Build faster with pre-made components |
| Easy Collaboration | Share flows with team visually |
| Export Options | Deploy to production easily |

### When to Use Langflow

✅ **Good for:**
- Rapid prototyping
- Testing different agent architectures
- Teaching and learning agent concepts
- Building simple to moderate complexity flows
- Collaborating with non-technical team members

❌ **Not ideal for:**
- Very complex custom logic
- Performance-critical applications
- Highly specialized workflows
- Fine-grained control over every detail

### Comparison with Other Approaches

| Feature | Langflow | Code-First (LangChain) | CrewAI | LangGraph |
|---------|----------|------------------------|--------|----------|
| Visual Interface | ✅ Yes | ❌ No | ❌ No | ⚠️ Limited |
| Learning Curve | Low | High | Medium | Medium |
| Flexibility | Medium | Very High | Medium | High |
| Prototyping Speed | Very Fast | Slow | Fast | Medium |
| Production Ready | Yes | Yes | Yes | Yes |
| Debugging | Easy | Hard | Medium | Medium |

### Next Steps

1. **Experiment**: Build the example flows in Langflow
2. **Customize**: Create your own custom components
3. **Integrate**: Connect with your RAG service or APIs
4. **Deploy**: Export and deploy a production flow
5. **Learn More**: Explore Langflow documentation and community

### Additional Resources

- [Langflow Documentation](https://docs.langflow.org/)
- [Langflow GitHub](https://github.com/logspace-ai/langflow)
- [watsonx.ai Documentation](https://www.ibm.com/docs/en/watsonx-as-a-service)
- [LangChain Documentation](https://python.langchain.com/)

---

**Course**: Multi-Agent Systems with watsonx.ai  
**Lab**: 3.4 - Langflow Visual Agent Builder  
**Platform**: Compatible with Google Colab and local environments

## 14. Cleanup

When you're done, stop the Langflow server:

In [ ]:
# Stop Langflow server
try:
    langflow_proc.terminate()
    langflow_proc.wait(timeout=5)
    print("✓ Langflow server stopped successfully")
except:
    try:
        langflow_proc.kill()
        print("✓ Langflow server force stopped")
    except:
        print("⚠ Could not stop Langflow server automatically")
        print("You may need to stop it manually or restart your kernel")